In [237]:
import glob
import os.path
from pathlib import Path

from PIL import Image
from tqdm.notebook import tqdm

SOURCE_IMAGES_ROOT = '../Screenshots for app store/sources'
DESTINATION_IMAGES_ROOT = '../Screenshots for app store/generated'

BUTTONS = {
    'gamepad': {
        'path': '../Bundle/JSDPad/dPad-None@2x.png',
        'offset': (20, -20-150),  # +x = left, -x = right, +y = top, -y = bottom
        'size': (150, 150),
    },
    'esc': {
        'path': '../Screenshots for app store/esc button@2x.png',
        'offset': (-20-50, 20),
        'size': (50, 50),
    },
    'return': {
        'path': '../Screenshots for app store/return button@2x.png',
        'offset': (-20-50, -20-50),
        'size': (50, 50),
    },
    'tab': {
        'path': '../Screenshots for app store/tab button@2x.png',
        'offset': (20, 20),
        'size': (50, 50),
    },
}
DIAGONALS_TO_PPI_SCALE_WIDTH_HEIGHT = {
    '5.5': (401, 2, 1920, 1080),
    '6.5': (458, 3, 2688, 1242),
    '12.9': (264, 2, 2732, 2048),
}
OPACITY = round(.125 * 255)


def generate_images():
  for diagonal in tqdm(DIAGONALS_TO_PPI_SCALE_WIDTH_HEIGHT):
    source_image_paths = glob.glob(os.path.join(SOURCE_IMAGES_ROOT, diagonal, '*.png'))
    output_directory = Path(os.path.join(DESTINATION_IMAGES_ROOT, diagonal))
    output_directory.mkdir(parents=True, exist_ok=True)

    for source_image_path in tqdm(source_image_paths):
      image = generate_image(source_image_path, diagonal)
      destination_image_path = os.path.join(
        output_directory, os.path.basename(source_image_path))
      image.save(destination_image_path)


def generate_image(source_image_path, diagonal):
    bg_image = Image.open(source_image_path)
    dpi, scale, *size = DIAGONALS_TO_PPI_SCALE_WIDTH_HEIGHT[diagonal]

    for name, data in BUTTONS.items():
        _process_button(bg_image, data, size, scale)
    return bg_image


def _process_button(bg_image, image_data, size, scale):
  real_size = [d*scale for d in image_data['size']]
  image = Image.open(image_data['path']).resize(real_size).convert(mode='RGBA')
  actual_offset = [
    _calculate_actual_offset(o, s, scale)
    for (o, s) in zip(image_data['offset'], size)
  ]

  image_data = list(image.getdata())
  for i, pixel in enumerate(image_data):
    image_data[i] = pixel[:3] + (OPACITY, ) if pixel[3] > 0 else pixel
  image.putdata(image_data)

  bg_image.paste(image, box=actual_offset, mask=image)


def _calculate_actual_offset(synth_offset, bg_image_size, scale):
  """'+' because `synth_offset` is less than zero and will decrease value on sum."""
  scaled_offset = synth_offset * scale
  return scaled_offset if synth_offset > 0 else bg_image_size + scaled_offset


generate_images()